In [11]:
import sys
!{sys.executable} -m pip install deepchecks -U --quiet

In [13]:
from deepchecks.tabular.datasets.regression import wine_quality
data = wine_quality.load_data(data_format = 'Dataframe', as_train_test = False)
data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data['quality'], test_size = 0.2, random_state = 42)
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [20]:
from deepchecks.tabular import Dataset
train_ds = Dataset(X_train, label = y_train, cat_features = [])
test_ds = Dataset(X_test, label= y_test, cat_features = [])

In [24]:
from deepchecks.tabular.suites import model_evaluation
evaluation_suite = model_evaluation()
suite_result = evaluation_suite.run(train_ds, test_ds, gbr)
suite_result.show()

Model Evaluation Suite: 
 
 
 
 12/12 [Time: 00:30, Check=Model Inference Time]

Accordion(children=(VBox(children=(HTML(value='\n            <h1 id="summary_0MZX05A8KJCA3UBFSQ0LGY8KA">Model …

In [25]:
from deepchecks.tabular.checks import TrainTestPerformance
gbr = GradientBoostingRegressor(n_estimators= 20)
gbr.fit(X_train, y_train)
check = TrainTestPerformance().add_condition_train_test_relative_degradation_less_than(0.3)
result = check.run(train_ds, test_ds, gbr)
result.show()

In [26]:
evaluation_suite

Model Evaluation Suite: [
	0: TrainTestPerformance
		Conditions:
			0: Train-Test scores relative degradation is less than 0.1
	1: RocReport(excluded_classes=[])
		Conditions:
			0: AUC score for all the classes is greater than 0.7
	2: ConfusionMatrixReport
	3: TrainTestPredictionDrift
		Conditions:
			0: categorical drift score < 0.15 and numerical drift score < 0.075
	4: SimpleModelComparison
		Conditions:
			0: Model performance gain over simple model is greater than 10%
	5: WeakSegmentsPerformance(n_to_show=5)
		Conditions:
			0: The relative performance of weakest segment is greater than 80% of average model performance.
	6: CalibrationScore
	7: RegressionSystematicError
		Conditions:
			0: Bias ratio is less than 0.01
	8: RegressionErrorDistribution
		Conditions:
			0: Kurtosis value is greater than -0.1
	9: UnusedFeatures
		Conditions:
			0: Number of high variance unused features is less or equal to 5
	10: BoostingOverfit
		Conditions:
			0: Test score over iterations is less t

In [27]:
evaluation_suite[0].clean_conditions()
evaluation_suite[0].add_condition_train_test_relative_degradation_less_than(0.3)
evaluation_suite = evaluation_suite.remove(7)

In [45]:
result = evaluation_suite.run(train_ds, test_ds, gbr)
result.passed(fail_if_warning= False)

Model Evaluation Suite: 
 
 
 
 8/8 [Time: 00:27, Check=Model Inference Time]

True

In [47]:
evaluation_suite[0].clean_conditions()
evaluation_suite[0].add_condition_train_test_relative_degradation_less_than(0.3)
evaluation_suite = evaluation_suite.remove(7)

AttributeError: 'Suite' object has no attribute 'clean_conditions'

In [1]:
result = evaluation_suite.run(train_ds, test_ds, gbr)
result.passed(fail_if_warning= False)

NameError: name 'evaluation_suite' is not defined